In [ ]:
import sys
import os

py_file_location = "Dirección archivos.py AdapNet++"
sys.path.append(os.path.abspath(py_file_location))

import argparse
import datetime
import importlib
import os
import re
import numpy as np
import tensorflow as tf
import yaml
from dataset.helper import *

In [ ]:
num_clases='Número de clases'
max_it='Número de iteraciones'
batch_size='Tamño de lote'

In [ ]:
def get_train_batch():
    filenames = ['Dirección tfrecords imágenes y etiquetas']
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.map(lambda x: parser(x, num_clases))
    dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.repeat(100)
    dataset = dataset.prefetch(1)
    iterator = dataset.make_one_shot_iterator()
    return iterator

def get_train_data():
    iterator = get_train_batch()
    dataA, label, dataB = iterator.get_next()
    return [dataA, label, dataB], iterator

data_list, iterator = get_train_data()

In [ ]:
module = importlib.import_module('models.'+'SSMA')
model_func = getattr(module,'SSMA')

global_step = tf.Variable(0, trainable=False, name='Global_Step')

model = model_func(num_classes=num_clases, learning_rate=0.001, decay_steps=max_it,power=0.0001, global_step=global_step)
images_pl = tf.placeholder(tf.float32, [None, 384, 768, 3])
images_pl1 = tf.placeholder(tf.float32, [None, 384, 768, 3])
labels_pl = tf.placeholder(tf.float32, [None, 384, 768, num_clases])
model.build_graph(images_pl, images_pl1, labels_pl)
model.create_optimizer()
config1 = tf.ConfigProto()
config1.gpu_options.allow_growth = True
sess = tf.Session(config=config1)
sess.run(tf.global_variables_initializer())
step = 'Iteración a partir de la que se inicia el fine-tuning'
total_loss = 0.0
t0 = None
path='Dirección checkpoint'
import_variables = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)

initialize_variables={}
trainable_collection=tf.get_collection_ref(tf.GraphKeys.TRAINABLE_VARIABLES)
#Lista con las capas entrenables
names=[]
for item in trainable_collection:
    temp=item.name.split(':')
    names.append(temp[0])

for var in import_variables:
  temp=var.name.split(':')
  if temp[0] not in names:
    trainable_collection.append(var)
    
    
saver=tf.train.Saver(import_variables)
saver.restore(sess,path)
#Número de checkpoints a guardar
saver = tf.train.Saver(max_to_keep=1500)

In [ ]:
perdidas=[]
while 1:
      try: 
          img, label, img1 = sess.run([data_list[0], data_list[1], data_list[2]]) 
          feed_dict = {images_pl: img, labels_pl: label, images_pl1: img1}
          loss_batch, _ = sess.run([model.loss, model.train_op],feed_dict=feed_dict)
          total_loss += loss_batch
        #Se guarda un checkpoint cada 10 iteraciones
          if (step + 1) % 10 == 0:
              saver.save(sess, os.path.join('Dirección para guardar checkpoint', 'nombre del checkpoint'), step)

          if (step + 1) % 10 == 0:
              left_hours = 0

              if t0 is not None:
                  delta_t = (datetime.datetime.now() - t0).seconds
                    #left_time=(num_iteraciones-step)/10-delta_t
                  left_time = (max_it - step) / 10 * delta_t
                  left_hours = left_time/3600.0

              t0 = datetime.datetime.now()
            #Media de las pérdidas a lo largo de las 10 iteraciones
              total_loss /= 10
              perdidas.append(total_loss)
              print ('%s %s] Step %s, lr = %f ' \
                % (str(datetime.datetime.now()), str(os.getpid()), step,
                    model.lr.eval(session=sess)))
              print ('\t loss = %.4f' % (total_loss))
              print ('\t estimated time left: %.1f hours. %d/%d' % (left_hours, step,max_it))
              print ('\t', 'SSMA')
              total_loss = 0.0

          step += 1

          if step > max_it:
              saver.save(sess, os.path.join('Dirección para guardar checkpoint', 'nombre del checkpoint'), step-1)
              print ('training_completed')
              break

      except tf.errors.OutOfRangeError:
          print ('Epochs in dataset repeat < max_iteration')
          break